# Search through gnps libraries to select dataset

Starting from gnps public libraries (speclibs folders, version from 5 July 2019), we here create one large MGF file with all spectra.
In addition Smiles or Inchi's are added from metadata file.

In [1]:
# Import
import json
import os

In [2]:
# Locations
ROOT = "C:\\OneDrive - Netherlands eScience Center\\Project_Wageningen_iOMEGA"
PATH_MS_DATA = ROOT + "\\Data\\GNPS_all\\"
PATH_SAVE_MODEL = ROOT + "\\Spec2Vec\\models_trained\\"
PATH_SAVE_DATA = ROOT + "\\Spec2Vec\\data\\"
PATH_SPEC2VEC = ROOT + "\\Spec2Vec\\"

## Importing additional metadata

In [3]:
json_file = PATH_MS_DATA + 'gnpslibraryjson_filtered.json'

with open(json_file, 'rb') as f:
    data = json.load(f)

In [4]:
print("Metadata available for", len(data), "spectra.")

Metadata available for 77253 spectra.


In [5]:
gnps_data = {}
for data_entry in data:
    if data_entry['GNPSID'] in gnps_data:
        print('duplicate!')
    gnps_data[data_entry['GNPSID']] = data_entry

## Briefly inspect metadata

In [6]:
keys = []
for key in gnps_data.keys():
    keys.append(key)

In [7]:
keys[0]

'CCMSLIB00000001548'

In [8]:
gnps_data[keys[0]]

{'COMPOUND_INCHIKEY': 'KNGPFNUOXXLKCN-ZNCJFREWSA-N',
 'COMPOUND_INCHI': 'InChI=1S/C45H73N5O10S3/c1-14-17-24(6)34(52)26(8)37-25(7)30(58-13)18-31-46-29(19-61-31)39-49-45(12,21-62-39)43-50-44(11,20-63-43)42(57)48-32(22(4)15-2)35(53)27(9)40(55)59-36(23(5)16-3)38(54)47-33(28(10)51)41(56)60-37/h19,22-28,30,32-37,51-53H,14-18,20-21H2,1-13H3,(H,47,54)(H,48,57)/t22-,23-,24+,25-,26-,27+,28+,30-,32-,33-,34-,35-,36-,37-,44+,45+/m0/s1',
 'LIBRARY_QUALITY': '1',
 'GNPSID': 'CCMSLIB00000001548',
 'COMPOUND_NAME': 'Hoiamide B',
 'COMPOUND_SMILES': 'CCC[C@@H](C)[C@@H]([C@H](C)[C@@H]1[C@H]([C@H](Cc2nc(cs2)C3=N[C@](CS3)(C4=N[C@](CS4)(C(=O)N[C@H]([C@H]([C@H](C(=O)O[C@H](C(=O)N[C@H](C(=O)O1)[C@@H](C)O)[C@@H](C)CC)C)O)[C@@H](C)CC)C)C)OC)C)O'}

## Functions

In [9]:
def check_spectrum_lines(spectrum):
    """ Check if desired metadata is present for spectrum.
    """
    spec_id = None
    smiles = None
    
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spec_id = l.strip().split('=')[1]
        if l.startswith('SMILES'):
            smiles = l.strip().split('=')[1]
    
    if spec_id in gnps_data:
        return True
    else:
        return False
    if smiles is not None \
        and smiles.strip() != 'N/A' \
        and smiles.strip() != 'NA'\
        and len(smiles) > 0 \
        and smiles.strip() != 'InChI':
        return True

    return False

def augment_spectrum(spectrum):
    """ Add metadata to spectrum (taken from json file, see above).
    """
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spectrum_id = l.strip().split('=')[1]
                
    new_spectrum = []
    for l in spectrum:
        if l.startswith('BEGIN ION'):
            new_spectrum.append(l)
            new_spectrum.append('TITLE=%s\n' % gnps_data[spectrum_id]['COMPOUND_NAME'])
            new_spectrum.append('LIBRARYQUALITY=%s\n' % gnps_data[spectrum_id]['LIBRARY_QUALITY'])
            new_spectrum.append('SMILES=%s\n' % gnps_data[spectrum_id]['COMPOUND_SMILES'])
            new_spectrum.append('INCHIKEY=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHIKEY'])
            new_spectrum.append('INCHI=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHI'])
            continue
            
        if l.startswith('INCHIKEY'):
            continue
        elif l.startswith('SMILES'):
            continue
        elif l.startswith('INCHI'):
            continue
        elif l.startswith('LIBRARYQUALITY'):
            continue
        elif l.startswith('TITLE'):
            continue
            
        new_spectrum.append(l)

    return new_spectrum

def match_spectra_in_file(filename, matching=True):
    """ Collect spectra in file (line by line). Check if matching metadata is available.
    
    matching: bool
        If matching = True than only include spectra for which desired metadata is present.
        For matching = False, include all spectra.
    """
    matched_spectra = []
    with open(filename) as f:
        for line in f.readlines():
            if line.startswith('BEGIN ION'):
                # Fresh spectrum
                spectrum = []
            spectrum.append(line)
            
            if line.startswith('END ION'):
                
                if check_spectrum_lines(spectrum): # Check if we have structural info
                    try:
                        spectrum = augment_spectrum(spectrum)
                    except KeyError as e:
                        pass
                    matched_spectra.append(spectrum)
                else: 
                    if not matching: # then include all spectra
                        matched_spectra.append(spectrum)
    return matched_spectra

## Create MGF for all spectra (also non matched ones)

In [10]:
path = PATH_MS_DATA + 'speclibs\\'
collected_spectra = []
for d in os.listdir(path):
    for f in os.listdir(os.path.join(path, d)):
        if not f.endswith('.mgf'):
            continue
        sp = match_spectra_in_file(os.path.join(path, d, f), matching=False)
        collected_spectra.extend(sp)
        print(f, len(collected_spectra))

BILELIB19.mgf 177
CASMI.mgf 745
DEREPLICATOR_IDENTIFIED_LIBRARY.mgf 1124
GNPS-COLLECTIONS-MISC.mgf 1170
GNPS-COLLECTIONS-PESTICIDES-NEGATIVE.mgf 1246
GNPS-COLLECTIONS-PESTICIDES-POSITIVE.mgf 1899
GNPS-EMBL-MCF.mgf 2484
GNPS-FAULKNERLEGACY.mgf 2611
GNPS-LIBRARY.mgf 7394
GNPS-NIH-CLINICALCOLLECTION1.mgf 7771
GNPS-NIH-CLINICALCOLLECTION2.mgf 7966
GNPS-NIH-NATURALPRODUCTSLIBRARY.mgf 9233
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_NEGATIVE.mgf 11096
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE.mgf 16892
GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE.mgf 18352
GNPS-NIST14-MATCHES.mgf 24115
GNPS-PRESTWICKPHYTOCHEM.mgf 24258
GNPS-SELLECKCHEM-FDA-PART1.mgf 26646
GNPS-SELLECKCHEM-FDA-PART2.mgf 27302
HMDB.mgf 29537
LDB_NEGATIVE.mgf 29763
LDB_POSITIVE.mgf 29846
MASSBANK.mgf 41845
MASSBANKEU.mgf 43337
MIADB.mgf 43509
MMV_NEGATIVE.mgf 43556
MMV_POSITIVE.mgf 43666
MONA.mgf 92907
PNNL-LIPIDS-NEGATIVE.mgf 109049
PNNL-LIPIDS-POSITIVE.mgf 139631
RESPECT.mgf 146743
SUMNER.mgf 147647


In [11]:
len(collected_spectra)

147647

# Convert smiles into inchi where necessary

In [23]:
from rdkit import Chem

def smiles_to_inchi(smiles):
    mol = Chem.MolFromSmiles(smiles)
    try: 
        inchi = Chem.inchi.MolToInchi(mol)
    except:
        print("Error for smiles:", smiles)
        inchi = "N/A"
    return inchi

def inchi_to_inchikey(inchi):
    try:
        inchikey = Chem.inchi.InchiToInchiKey(inchi)
    except:
        print("Error for inchi to inchikey:", inchi)
        inchikey = "N/A"   
    return inchikey

In [13]:
# to "save" for next testing 
#collected_spectra_save = collected_spectra.copy()

In [25]:
collected_spectra = collected_spectra_save.copy()

### Make inchi strings more homogeneous in style

In [26]:
for i, spectrum in enumerate(collected_spectra):
    inchi = 'N/A'
    
    spectrum_new = []
    for l in spectrum:
        if l.startswith('INCHI='):
            inchi = l.strip().split('INCHI=')[1]
            
            if inchi is None \
                or inchi in ['N/A', 'NA', 0, '0'] \
                or len(inchi) == 0:
                continue
            else:
                if (inchi.startswith('"InChI=') and inchi.endswith('"')):
                    pass
                elif (inchi.startswith('InChI=')):
                    inchi = '"' + inchi + '"'
                elif inchi.endswith('"'):
                    inchi = '"InChI=' + inchi
                else:
                    inchi = '"InChI=' + inchi + '"'
                l = 'INCHI=%s\n' % inchi
        spectrum_new.append(l)
    collected_spectra[i] = spectrum_new 

In [21]:
collected_spectra[0]

['BEGIN IONS\n',
 'PEPMASS=391.285\n',
 'CHARGE=1\n',
 'MSLEVEL=2\n',
 'SOURCE_INSTRUMENT=ESI-Orbitrap\n',
 'FILENAME=Standards_n_1ugmL_chenodeoxycholic.mzXML\n',
 'SEQ=*..*\n',
 'IONMODE=Negative\n',
 'ORGANISM=BILELIB19\n',
 'NAME=chenodeoxycholic acid M-H\n',
 'PI=Dorrestein\n',
 'DATACOLLECTOR=A. Jarmusch\n',
 'SMILES=C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2[C@@H](C[C@H]4[C@@]3(CC[C@H](C4)O)C)O)C\n',
 'INCHI="InChI=1S/C24H40O4/c1-14(4-7-21(27)28)17-5-6-18-22-19(9-11-24(17,18)3)23(2)10-8-16(25)12-15(23)13-20(22)26/h14-20,22,25-26H,4-13H2,1-3H3,(H,27,28)/t14?,15-,16+,17+,18-,19-,20+,22-,23?,24?/m0/s1"\n',
 'INCHIAUX=N/A\n',
 'PUBMED=N/A\n',
 'SUBMITUSER=mpanitchpakdi\n',
 'TAGS=\n',
 'LIBRARYQUALITY=1\n',
 'SPECTRUMID=CCMSLIB00005435444\n',
 'SCANS=1688\n',
 '51.787151\t18114.681641\n',
 '58.22208\t18008.097656\n',
 '71.187103\t21309.207031\n',
 '75.990234\t20840.488281\n',
 '80.070091\t20326.140625\n',
 '91.880928\t23875.916016\n',
 '95.083244\t23740.320312\n',
 '105.069

### Search for spectra with smiles but no inchi (and try to convert smiles to inchi)

In [29]:
inchi_lst = []
inchikey_lst = []
smiles_lst = []

conv = ob.OBConversion()
conv.SetInAndOutFormats("inchi", "inchikey")
mol = ob.OBMol()

changes_to_spectra = []
for i, spectrum in enumerate(collected_spectra):
    smiles = 'N/A'
    inchi = 'N/A'
    inchikey = 'N/A'
    modified = False
    #spectrum_new = []
    
    inchi_line = 0
    inchikey_line = 0
    inchiaux_line = 0
    for m, l in enumerate(spectrum):
        if l.startswith('SMILES'):
            smiles = l.strip().split('SMILES=')[1]
        if l.startswith('INCHI='):
            inchi = l.strip().split('INCHI=')[1]
            inchi_line = m
        if l.startswith('INCHIKEY='):
            inchikey = l.strip().split('INCHIKEY=')[1]
            inchikey_line = m
        if l.startswith('INCHIAUX='):
            inchiaux_line = m
        if inchikey_line == 0:
            if inchiaux_line == 0:
                spectrum.insert(inchikey_line+1, 'INCHIKEY=N/A\n')
            else:
                spectrum[inchiaux_line] = 'INCHIKEY=N/A\n'           
        
    # Add to lists
    smiles_lst.append(smiles)
    inchi_lst.append(inchi)
    inchikey_lst.append(inchikey)
    
    # Look for cases with smiles BUT NO inchi 
    if inchi is None \
        or inchi in ['N/A', 'NA', 0, '0'] \
        or len(inchi) == 0:
            if smiles is not None \
                and smiles not in ['N/A', 'NA', 0, '0', 'InChI'] \
                and len(smiles) > 0:  
                    for m, l in enumerate(spectrum):
                        if l.startswith('INCHI='):
                            inchi = smiles_to_inchi(smiles)
                            l = 'INCHI=%s\n' % inchi
                            spectrum[m] = l
                            modified = True
                            inchi_lst[-1] = inchi 
                            continue #assuming there is only one line with inchi

    # Look for missing inchikeys
    if inchikey is None \
        or inchikey in ['N/A', 'NA', 0, '0'] \
        or len(inchikey) == 0:
            if len(inchi) > 5:
                conv.ReadString(mol, inchi_lst[741])
                found_inchikey = conv.WriteString(mol)
                #found_inchikey = inchi_to_inchikey(inchi)
                for m, l in enumerate(spectrum):
                    if l.startswith('INCHIKEY='):
                        l = 'INCHIKEY=%s\n' % found_inchikey
                        spectrum[m] = l
                        modified = True
                        inchikey_lst[-1] = found_inchikey

    # Replace spectrum with modified one                                 
    if modified:                                       
        collected_spectra[i] = spectrum                  
        #print("Added inchi or inchikey to spectrum", i, inchi_lst[-1], inchikey_lst[-1])
        changes_to_spectra.append((i, inchi_lst[-1], inchikey_lst[-1]))

In [52]:
inchi_lst = []
inchikey_lst = []
smiles_lst = []

inchi_to_keys = ob.OBConversion()
inchi_to_keys.SetInAndOutFormats("inchi", "inchikey")
mol = ob.OBMol()
smiles_to_inchi = ob.OBConversion()
smiles_to_inchi.SetInAndOutFormats("smi", "inchi")
#conv.SetOptions("K", conv.OUTOPTIONS)

changes_to_spectra = []
for i, spectrum in enumerate(collected_spectra):
    smiles = 'N/A'
    inchi = 'N/A'
    inchikey = 'N/A'
    modified = False
    #spectrum_new = []
    
    inchi_line = 0
    inchikey_line = 0
    inchiaux_line = 0
    for m, l in enumerate(spectrum):
        if l.startswith('SMILES'):
            smiles = l.strip().split('SMILES=')[1]
        if l.startswith('INCHI='):
            inchi = l.strip().split('INCHI=')[1]
            inchi_line = m
        if l.startswith('INCHIKEY='):
            inchikey = l.strip().split('INCHIKEY=')[1]
            inchikey_line = m
        if l.startswith('INCHIAUX='):
            inchiaux_line = m
    if inchikey_line == 0: # Add inchikey line if not present
        if inchiaux_line == 0:
            spectrum.insert(inchi_line+1, 'INCHIKEY=N/A\n')
        else:
            spectrum[inchiaux_line] = 'INCHIKEY=N/A\n'           
        
    # Add to lists
    smiles_lst.append(smiles)
    inchi_lst.append(inchi)
    inchikey_lst.append(inchikey)
    
    # Look for cases with smiles BUT NO inchi 
    if inchi is None \
        or inchi in ['N/A', 'NA', 0, '0'] \
        or len(inchi) == 0:
            if smiles is not None \
                and smiles not in ['N/A', 'NA', 0, '0', 'InChI'] \
                and len(smiles) > 0:  
                    for m, l in enumerate(spectrum):
                        if l.startswith('INCHI='):
                            #inchi = smiles_to_inchi(smiles)
                            mol = ob.OBMol()
                            smiles_to_inchi.ReadString(mol, smiles)
                            inchi = smiles_to_inchi.WriteString(mol)
                            l = 'INCHI=%s\n' % inchi
                            spectrum[m] = l
                            modified = True
                            inchi_lst[-1] = inchi 
                            continue #assuming there is only one line with inchi

    # Look for missing inchikeys
    if inchikey is None \
        or inchikey in ['N/A', 'NA', 0, '0'] \
        or len(inchikey) == 0:
            if len(inchi) > 5:
                inchi_to_keys.ReadString(mol, inchi)
                found_inchikey = inchi_to_keys.WriteString(mol)

                for m, l in enumerate(spectrum):
                    if l.startswith('INCHIKEY='):
                        l = 'INCHIKEY=%s\n' % found_inchikey
                        spectrum[m] = l
                        modified = True
                        inchikey_lst[-1] = found_inchikey

    # Replace spectrum with modified one                                 
    if modified:                                       
        collected_spectra[i] = spectrum                  
        #print("Added inchi or inchikey to spectrum", i, inchi_lst[-1], inchikey_lst[-1])
        changes_to_spectra.append((i, inchi_lst[-1], inchikey_lst[-1]))

In [55]:
collected_spectra[0]

['BEGIN IONS\n',
 'INCHIKEY=N/A\n',
 'INCHIKEY=N/A\n',
 'INCHIKEY=None\n',
 'PEPMASS=391.285\n',
 'CHARGE=1\n',
 'MSLEVEL=2\n',
 'SOURCE_INSTRUMENT=ESI-Orbitrap\n',
 'FILENAME=Standards_n_1ugmL_chenodeoxycholic.mzXML\n',
 'SEQ=*..*\n',
 'IONMODE=Negative\n',
 'ORGANISM=BILELIB19\n',
 'NAME=chenodeoxycholic acid M-H\n',
 'PI=Dorrestein\n',
 'DATACOLLECTOR=A. Jarmusch\n',
 'SMILES=C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2[C@@H](C[C@H]4[C@@]3(CC[C@H](C4)O)C)O)C\n',
 'INCHI="InChI=1S/C24H40O4/c1-14(4-7-21(27)28)17-5-6-18-22-19(9-11-24(17,18)3)23(2)10-8-16(25)12-15(23)13-20(22)26/h14-20,22,25-26H,4-13H2,1-3H3,(H,27,28)/t14?,15-,16+,17+,18-,19-,20+,22-,23?,24?/m0/s1"\n',
 'INCHIAUX=N/A\n',
 'PUBMED=N/A\n',
 'SUBMITUSER=mpanitchpakdi\n',
 'TAGS=\n',
 'LIBRARYQUALITY=1\n',
 'SPECTRUMID=CCMSLIB00005435444\n',
 'SCANS=1688\n',
 '51.787151\t18114.681641\n',
 '58.22208\t18008.097656\n',
 '71.187103\t21309.207031\n',
 '75.990234\t20840.488281\n',
 '80.070091\t20326.140625\n',
 '91.880928

In [54]:
inchikey_lst[:20], smiles_lst[:20]

(['None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None',
  'None'],
 ['C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2[C@@H](C[C@H]4[C@@]3(CC[C@H](C4)O)C)O)C',
  'C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2[C@@H](C[C@H]4[C@@]3(CC[C@H](C4)O)C)O)C',
  'C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2[C@@H](C[C@H]4[C@@]3(CC[C@H](C4)O)C)O)C',
  'C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1([C@H](C[C@H]3[C@H]2[C@@H](C[C@H]4[C@@]3(CC[C@H](C4)O)C)O)O)C',
  'C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1([C@H](C[C@H]3[C@H]2[C@@H](C[C@H]4[C@@]3(CC[C@H](C4)O)C)O)O)C',
  'C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1([C@H](C[C@H]3[C@H]2[C@@H](C[C@H]4[C@@]3(CC[C@H](C4)O)C)O)O)C',
  'C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1([C@H](C[C@H]3[C@H]2CC[C@H]4[C@@]3(CC[C@H](C4)O)C)O)C',
  'C[C@H](CCC(=O)O)[C@H]1CC[C@@H]2[C@@]1([C@H](C[C@H]3[C@H]2CC[C@H]4[C@@]3(CC[C@H](C4)O)C)O)C',
  'C

In [51]:
inchi_lst[:20], smiles_lst[:20]

(['"InChI=1S/C24H40O4/c1-14(4-7-21(27)28)17-5-6-18-22-19(9-11-24(17,18)3)23(2)10-8-16(25)12-15(23)13-20(22)26/h14-20,22,25-26H,4-13H2,1-3H3,(H,27,28)/t14?,15-,16+,17+,18-,19-,20+,22-,23?,24?/m0/s1"',
  '"InChI=1S/C24H40O4/c1-14(4-7-21(27)28)17-5-6-18-22-19(9-11-24(17,18)3)23(2)10-8-16(25)12-15(23)13-20(22)26/h14-20,22,25-26H,4-13H2,1-3H3,(H,27,28)/t14?,15-,16+,17+,18-,19-,20+,22-,23?,24?/m0/s2"',
  '"InChI=1S/C24H40O4/c1-14(4-7-21(27)28)17-5-6-18-22-19(9-11-24(17,18)3)23(2)10-8-16(25)12-15(23)13-20(22)26/h14-20,22,25-26H,4-13H2,1-3H3,(H,27,28)/t14?,15-,16+,17+,18-,19-,20+,22-,23?,24?/m0/s2"',
  '"InChI=1S/C24H40O5/c1-13(4-7-21(28)29)16-5-6-17-22-18(12-20(27)24(16,17)3)23(2)9-8-15(25)10-14(23)11-19(22)26/h13-20,22,25-27H,4-12H2,1-3H3,(H,28,29)/t13?,14-,15+,16+,17-,18-,19+,20-,22-,23?,24?/m0/s1"',
  '"InChI=1S/C24H40O5/c1-13(4-7-21(28)29)16-5-6-17-22-18(12-20(27)24(16,17)3)23(2)9-8-15(25)10-14(23)11-19(22)26/h13-20,22,25-27H,4-12H2,1-3H3,(H,28,29)/t13?,14-,15+,16+,17-,18-,19+,20-,22-,23?

In [32]:
inchi_lst[740],inchi_lst[70]

('"InChI=1S/C24H27NO2/c1-3-5-12-19(4-2)18-27-24(26)22(17-25)23(20-13-8-6-9-14-20)21-15-10-7-11-16-21/h6-11,13-16,19H,3-5,12,18H2,1-2H3"',
 '"InChI=1S/C26H43NO6/c1-14(4-7-22(31)27-13-23(32)33)17-5-6-18-24-19(12-21(30)26(17,18)3)25(2)9-8-16(28)10-15(25)11-20(24)29/h14-21,24,28-30H,4-13H2,1-3H3,(H,27,31)(H,32,33)/t14?,15-,16+,17+,18-,19-,20+,21-,24-,25?,26?/m0/s1"')

In [34]:
import openbabel as ob
from openbabel import pybel

moltest = pybel.readstring("inchi", inchi_lst[70])

In [35]:
print(inchi_to_inchikey(inchi_lst[70]))

None


In [51]:
conv = ob.OBConversion()
conv.SetInAndOutFormats("smi", "inchi")
conv.SetOptions("K", conv.OUTOPTIONS)

mol = ob.OBMol()
conv.ReadString(mol, "CC(=O)Cl")
inchikey = conv.WriteString(mol)
print(inchikey)

WETWJCDKMRHUPV-UHFFFAOYSA-N



In [49]:
smiles_to_inchi = ob.OBConversion()
smiles_to_inchi.SetInAndOutFormats("smi", "inchi")
#conv.SetOptions("K", conv.OUTOPTIONS)

mol = ob.OBMol()
smiles_to_inchi.ReadString(mol, smiles_lst[71])
smiles = smiles_to_inchi.WriteString(mol)
print(smiles)

InChI=1S/C26H43NO6/c1-14(4-7-22(31)27-13-23(32)33)17-5-6-18-24-19(12-21(30)26(17,18)3)25(2)9-8-16(28)10-15(25)11-20(24)29/h14-21,24,28-30H,4-13H2,1-3H3,(H,27,31)(H,32,33)/t14-,15+,16-,17-,18+,19+,20-,21+,24+,25+,26-/m1/s1



In [44]:
conv = ob.OBConversion()
conv.SetInAndOutFormats("inchi", "inchikey") #"smi")
#conv.SetOptions("K", conv.OUTOPTIONS)

mol = ob.OBMol()
conv.ReadString(mol, inchi_lst[741])
inchikey = conv.WriteString(mol)
print(inchikey)

AFDXODALSZRGIH-QPJJXVBHSA-N



In [47]:
conv.ReadString(mol, inchi_lst[20])
inchikey = conv.WriteString(mol)
print(inchikey)

ZQYUKJFJPJDMMR-ZDWCHQGWSA-N



In [88]:

smiles = ['CCCC', 'CCCN']
mols = [pybel.readstring("smi", x) for x in smiles] # Create a list of two˓→molecules

fps = [x.calcfp() for x in mols]# Calculate their fingerprints
print(fps[0].bits, fps[1].bits)
print(fps[0] | fps[1])# Print the Tanimoto coefficient

[261, 385, 671] [83, 261, 349, 671, 907]
0.3333333333333333


In [98]:
#miles = ['CCCC', 'CCCN']
mols = [pybel.readstring("inchi", "InChI="+ x) for x in inchi_lst[:15]] #list of two˓→molecules

fps = [x.calcfp() for x in mols]# Calculate their fingerprints
print(fps[0].bits, fps[14].bits)
print(fps[1] | fps[14])# Print the Tanimoto coefficient

[50, 89, 261, 313, 330, 332, 385, 430, 439, 442, 516, 532, 600, 602, 612, 624, 635, 653, 671, 884, 985, 1009] [25, 50, 83, 89, 155, 191, 261, 268, 307, 313, 314, 330, 332, 337, 350, 353, 385, 402, 404, 430, 432, 434, 439, 442, 492, 516, 532, 544, 600, 602, 624, 641, 654, 671, 689, 705, 734, 782, 829, 884, 904, 907, 908, 916, 950, 980, 985, 994]
0.34615384615384615


In [67]:
collected_spectra_save[143150]

['BEGIN IONS\n',
 'PEPMASS=507.43\n',
 'CHARGE=1\n',
 'MSLEVEL=2\n',
 'SOURCE_INSTRUMENT=ESI-Flow-injection QqQ/MS\n',
 'FILENAME=respect_8_1_2014_GNPS_peaks.mgf\n',
 'SEQ=*..*\n',
 'IONMODE=Positive\n',
 'ORGANISM=RESPECT\n',
 "NAME=ReSpect:PS043401 Quercetin-3-O-glucose-6''-acetate|Quer-3-Glc-6pp-acetate|quercetin-3-O-beta-D-glucopyranosyl-6''-acetate|quercetin 3-(6''-acetylglucoside) [M+H]\n",
 'PI=Putative ReSpect Match\n',
 'DATACOLLECTOR=ReSpect\n',
 'SMILES=no data\n',
 'INCHI=N/A\n',
 'INCHIAUX=N/A\n',
 'PUBMED=N/A\n',
 'SUBMITUSER=mwang87\n',
 'TAGS=\n',
 'LIBRARYQUALITY=3\n',
 'SPECTRUMID=CCMSLIB00000215686\n',
 'SCANS=3522\n',
 '303.0\t87.0\n',
 '506.0\t275.0\n',
 '507.0\t999.0\n',
 '508.0\t299.0\n',
 'END IONS\n']

In [63]:
import sys

file_mgf = PATH_MS_DATA + 'all_gnps_inchi_added_191023.mgf'
with open(file_mgf, 'w') as f:
    for c in collected_spectra[:]:
        for l in c:
            try:
                f.write(l.encode('utf8', 'replace').decode('utf8', 'replace'))
            except UnicodeEncodeError:
                f.write(l.encode('ascii', 'replace').decode('utf8', 'replace'))
                print(l)
        f.write('\n')

TITLE=C16 Galactosyl(�) Ceramide (d18:1/16:0)

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8

TITLE=(1S,8S,9S,10S,13R)-6,9,10-trimethyl-2-oxo-4,14-dioxatetracyclo[7.5.0.0�,��.0�,?]tetradeca-3(7),5-dien-8-yl acetate

TITLE=(1S,8S,9S,10S,13R)-6,9,10-trimethyl-2-oxo-4,14-dioxatetracyclo[7.5.0.0�,��.0�,?]tetradeca-3(7),5-dien-8-yl acetate

TITLE=(4E)-8-hydroxy-4-(1-hydroxypropan-2-ylidene)-10-oxatricyclo[7.2.1.0�,?]dodecane-8-carboxylic acid

TITLE=(4E)-8-hydroxy-4-(1-hydroxypropan-2-ylidene)-10-oxatricyclo[7.2.1.0�,?]dodecane-8-carboxylic acid

TITLE=(2R,3S,4S,9R,12R,13R,14S,15R,16R,17S)-3,4,12,13,15,16-hexahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-6-en-11-one

TITLE=(2R,3S,4S,9R,12R,13R,14S,15R,16R,17S)-3,4,12,13,15,16-hexahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-6-en-11-one

TITLE=15-(carbamoylmethyl)-10,11,23-trihydroxy-18-(3-methyl-2-oxopentanamido)-9,14,17-trioxo-N-[(1Z)-prop-1-en-1-yl]-8,13,16-triazatetracyclo[18.3.1.0�,?.0?,�?]tetracosa-1(23),2(7),3,5,20(24),21-hexaene-12-carboxamide

TITLE=15-(carbamoylmethyl

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4,5-trihydroxybenzoyloxy)-3,6,9,24,27,33-hexaoxaheptacyclo[28.7.1.0?,�?.0?,�?.0��,�?.0�?,��.0�?,�?]octatriaconta-1(38),11(16),12,14,17,19,21,34,36-nonaen-29-yl]acetic acid

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=1,17,19-trihydroxy-7-methyl-3-oxapentacyclo[9.8.0.0�,?.0?,�?.0��,�?]nonadeca-5(10),13,15,17-tetraene-9,12-dione

TITLE=2-[(5S,8R,11S,12S,13S,21R)-13,17,18-trihyd

...

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-10-oxatetracyclo[7.6.1.0�,?.0��,�?]hexadec-4-en-11-one

TITLE=(2R,3R,6R,8R,9S,12S,13R,14R,15R,16R)-6,8,14,15-tetrahydroxy-2,6,13,16-tetramethyl-3-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)o

TITLE=(1R,4S,5R,9R,10R,13S)-13-{[(2S,3R,4S,5R,6R)-5-hydroxy-6-(hydroxymethyl)-3,4-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})oxan-2-yl]oxy}-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(1R,4S,5R,9R,10R,13S)-13-{[(2S,3R,4S,5R,6R)-5-hydroxy-6-(hydroxymethyl)-3,4-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})oxan-2-yl]oxy}-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(1R,4S,5R,9R,10R,13S)-13-{[(2S,3R,4S,5R,6R)-5-hydroxy-6-(hydroxymethyl)-3,4-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})oxan-2-yl]oxy}-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(1R,4S,5R,9R,10R,13S)-13-{[(2S,3R,4S,5R,6R)-5-hydroxy-6-(hydroxymethyl)-3,4-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})oxan-2-yl]oxy}-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane

TITLE=(2S,3R,4R,5R,6S)-2-[(4-hydroxy-6-{[(4S,6R,7S,8R,9S,13S,16S)-6-hydroxy-7,9,13-trimethyl-6-[(3R)-3-methyl-4-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}butyl]-5-oxapentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan-16-yl]oxy}-2-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-3-yl)oxy]-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-[(4-hydroxy-6-{[(4S,6R,7S,8R,9S,13S,16S)-6-hydroxy-7,9,13-trimethyl-6-[(3R)-3-methyl-4-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}butyl]-5-oxapentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan-16-yl]oxy}-2-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-3-yl)oxy]-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-[(4-hydroxy-6-{[(4S,6R,7S,8R,9S,13S,16S)-6-hydroxy-7,9,13-trimethyl-6-[(3R)-3-methyl-4-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}butyl]-5-oxapentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan-16-yl]oxy}-2-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hy

TITLE=(2S,3R,4R,5R,6S)-2-{[(2R,3S,4R,5R,6S)-6-{[(2R,3R,4S,5S,6R)-4,5-dihydroxy-6-(hydroxymethyl)-2-[(1'S,2S,4'S,5S,7'R,9'S,13'R,16'S)-5,7',9',13'-tetramethyl-5-({[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}methyl)-5'-oxaspiro[oxolane-2,6'-pentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan]-18'-eneoxy]oxan-3-yl]oxy}-4,5-dihydroxy-2-(hydroxymethyl)oxan-3-yl]oxy}-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-{[(2R,3S,4R,5R,6S)-6-{[(2R,3R,4S,5S,6R)-4,5-dihydroxy-6-(hydroxymethyl)-2-[(1'S,2S,4'S,5S,7'R,9'S,13'R,16'S)-5,7',9',13'-tetramethyl-5-({[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}methyl)-5'-oxaspiro[oxolane-2,6'-pentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan]-18'-eneoxy]oxan-3-yl]oxy}-4,5-dihydroxy-2-(hydroxymethyl)oxan-3-yl]oxy}-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-{[(2R,3S,4R,5R,6S)-6-{[(2R,3R,4S,5S,6R)-4,5-dihydroxy-6-(hydroxymethyl)-2-[(1'S,2S,4'S,5S,7'R,9'S,13'R,16'S)-5,7',9',13'-tetramethyl-5-({[(2R,3R,4S,5S,6R)-3,4,5-trihydro

TITLE=(1S,2R,4S,9R,10R,14S,15S,17S)-9-(furan-3-yl)-1-hydroxy-15-[(1R)-1-hydroxy-2-methoxy-2-oxoethyl]-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo[12.3.1.0�,?.0?,��.0?,�?]octadecan-17-yl propanoate

TITLE=(1S,2R,4S,9R,10R,14S,15S,17S)-9-(furan-3-yl)-1-hydroxy-15-[(1R)-1-hydroxy-2-methoxy-2-oxoethyl]-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo[12.3.1.0�,?.0?,��.0?,�?]octadecan-17-yl propanoate

TITLE=(1S,2R,4S,9R,10R,14S,15S,17S)-9-(furan-3-yl)-1-hydroxy-15-[(1R)-1-hydroxy-2-methoxy-2-oxoethyl]-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo[12.3.1.0�,?.0?,��.0?,�?]octadecan-17-yl propanoate

TITLE=(1S,2R,4S,9R,10R,14S,15S,17S)-9-(furan-3-yl)-1-hydroxy-15-[(1R)-1-hydroxy-2-methoxy-2-oxoethyl]-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo[12.3.1.0�,?.0?,��.0?,�?]octadecan-17-yl 2-methylpropanoate

TITLE=(1S,2R,4S,9R,10R,14S,15S,17S)-9-(furan-3-yl)-1-hydroxy-15-[(1R)-1-hydroxy-2-methoxy-2-oxoethyl]-10,14,16,16-tetramethyl-7,18-dioxo-3,8-dioxapentacyclo

## Replace charge values of "-1+" with "-1"

In [64]:
input_file = PATH_MS_DATA + 'all_gnps_inchi_added_191023.mgf'
output_file = open(ROOT + "\\Data\\GNPS_all\\" + "MS_data_allGNPS_191023_corrected.mgf", 'w')

with open(input_file, 'r') as file:
    for line in file:
        output_file.write(line.replace('-1+', '-1'))

In [65]:
output_file.close()